In [ ]:
pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import patoolib

In [ ]:
patoolib.extract_archive('/content/archive (1).zip')

INFO patool: Extracting /content/archive (1).zip ...
INFO:patool:Extracting /content/archive (1).zip ...
INFO patool: running /usr/bin/7z x -aou -o./Unpack_x52hw28y -- "/content/archive (1).zip"
INFO:patool:running /usr/bin/7z x -aou -o./Unpack_x52hw28y -- "/content/archive (1).zip"
INFO patool: ... /content/archive (1).zip extracted to `donateacry_corpus'.
INFO:patool:... /content/archive (1).zip extracted to `donateacry_corpus'.


'donateacry_corpus'

In [ ]:
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import os

In [ ]:
data_dir = "/content/donateacry_corpus"
categories = ["belly_pain", "burping", "discomfort", "hungry", "tired"]

In [ ]:
data = []
labels = []

# Load audio data and label
def load_audio_data(data_dir, categories):
    for label, category in enumerate(categories):
        category_path = os.path.join(data_dir, category)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            try:
                # Load audio file and extract MFCC features
                audio, sr = librosa.load(file_path, sr=None)
                mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
                mfcc_scaled = np.mean(mfcc.T, axis=0)  # Take the mean across time
                data.append(mfcc_scaled)
                labels.append(label)
            except Exception as e:
                print(f"Error loading {file_name}: {e}")

In [ ]:
load_audio_data(data_dir, categories)

# Convert data to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

# Normalize the data
X_train = X_train / np.max(np.abs(X_train))
X_test = X_test / np.max(np.abs(X_test))

In [ ]:
X_train = X_train.reshape(-1, 40, 1)
X_test = X_test.reshape(-1, 40, 1)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(40, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.5860 - loss: 1.5655 - val_accuracy: 0.8370 - val_loss: 1.3095
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8089 - loss: 1.1505 - val_accuracy: 0.8370 - val_loss: 0.6811
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8239 - loss: 0.7207 - val_accuracy: 0.8370 - val_loss: 0.7099
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8465 - loss: 0.6443 - val_accuracy: 0.8370 - val_loss: 0.6658
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8166 - loss: 0.7202 - val_accuracy: 0.8370 - val_loss: 0.6594
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8501 - loss: 0.6254 - val_accuracy: 0.8370 - val_loss: 0.6625
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8210 - loss: 0.7117 - val_accuracy: 0.8370 - val_loss: 0.6617
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8206 - loss: 0.6979 - val_accuracy: 0.8370 - va

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=categories))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Classification Report:
               precision    recall  f1-score   support

  belly_pain       0.00      0.00      0.00         3
     burping       0.00      0.00      0.00         2
  discomfort       0.00      0.00      0.00         5
      hungry       0.84      1.00      0.91        77
       tired       0.00      0.00      0.00         5

    accuracy                           0.84        92
   macro avg       0.17      0.20      0.18        92
weighted avg       0.70      0.84      0.76        92

Accuracy Score: 0.8369565217391305


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
